In [7]:
# Example genserver generator using macros

defmodule Homebase.Services.Actor do

    defmacro __using__(opts \\ Keyword.new()) do
        quote do
            import Homebase.Services.Actor, only: [call: 3]
            use GenServer
            
            def start_link(opts \\ [name: __MODULE__]) do
                name = Keyword.get(opts, :name, __MODULE__)
                opts = Keyword.merge(unquote(opts), opts)
                if Keyword.get(opts, :name) == nil do
                    opts = Keyword.set(opts, :name, name)
                end
                
                GenServer.start_link(__MODULE__, initial_state, opts)
            end
        end
    end
    
    defmacro actor(name, do: block) do
        # IO.puts Macro.to_string(block)
        quote do
            defmodule unquote(name) do
               use Homebase.Services.Actor
               unquote(block)
            end
        end
    end
    
    defmacro call(msg, state, do: block) do
        fn_name = msg
        quote do
            def handle_call(unquote(msg), _from, unquote(state)) do
                res = unquote(block)
                if is_tuple(res) do
                    [:reply] ++ Tuple.to_list(res) |> List.to_tuple
                else
                    {:reply, res, res}
                end
                
                # db persis
            end
        end
    end

end

{:module, Homebase.Services.Actor, <<70, 79, 82, 49, 0, 0, 11, 188, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 1, 36, 0, 0, 0, 29, 30, 69, 108, 105, 120, 105, 114, 46, 72, 111, 109, 101, 98, 97, 115, 101, 46, 83, 101, 114, 118, 105, 99, ...>>, {:call, 3}}

In [2]:
# Using with idiomatic elixir module and use

defmodule ElixirActor do
    use Homebase.Services.Actor
    
    def initial_state, do: 20
    
    call(:test, state) do
        state + 10
    end
    
    call(:test2, state) do
        {state - 3, state}
    end
end

{:ok, pid} = ElixirActor.start_link()
IO.inspect GenServer.call(pid, :test)
IO.inspect GenServer.call(pid, :test2)
IO.inspect GenServer.call(pid, :test)

30
27
40


40

In [8]:
# Using top level macro, inspired by meh/amnesia
import Homebase.Services.Actor

actor Connor.Counter do

    def initial_state, do: 0

    call(:increment, state) do
        state + 1
    end
    
    call :decrement, state do
        state - 1
    end
    
    call {:set_value, value}, state, do: value
    
end

(
  def(initial_state) do
    0
  end
  call(:increment, state) do
    state + 1
  end
  call(:decrement, state) do
    state - 1
  end
  call({:set_value, value}, state) do
    value
  end
)


{:module, Connor.Counter, <<70, 79, 82, 49, 0, 0, 18, 112, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 2, 30, 0, 0, 0, 56, 21, 69, 108, 105, 120, 105, 114, 46, 67, 111, 110, 110, 111, 114, 46, 67, 111, 117, 110, 116, 101, 114, 8, ...>>, {:handle_call, 3}}

In [4]:
{:ok, pid} = Connor.Counter.start_link(name: :test_counter_1)
pid

GenServer.call(pid, :increment)
GenServer.call(pid, :increment)
GenServer.call(pid, :increment)
GenServer.call(pid, :increment)

4

In [5]:
# Making a python actor that is monitored and communicated with an otp actor
import Homebase.Services.Python

py_actor Connor.Counter do
    from homebase import handle_call
    count = 0

    @handle_call("increment")
    def increment():
        return count++
        
    @handle_call("decrement")
    def decrement():
        return count--

    @handle_call("set_value")
    def set_value(value):
        return count = value
end

MatchError: 1

In [10]:
Macro.to_string(quote(do: 1+1))

"1 + 1"